In [28]:
! pip install qdrant-client[fastembed]

In [29]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import os
import csv

In [30]:
books = [*csv.DictReader(open('/usr/local/dataset/dataset.csv'))]

In [31]:
documents: list[dict[str, any]] = []
metadata: list[dict[str, any]] = []
ids: list[int] = []

for idx, doc in enumerate(books):
    ids.append(idx)
    documents.append(doc["description"])
    metadata.append(
        {
            "title": doc["title"],
            "author": doc["author"],
            "publishDate": doc["publishDate"],
        }
    )

In [32]:
qdrant = QdrantClient(
    url="http://qdrant-database:6333", api_key=os.getenv("APIKEY"))

Create a Qdrant collection and insert data. This method establishes a connection to Qdrant, creates a new collection named `my_books`, and uploads the book data to `my_books`.

In [33]:
qdrant.add(collection_name="my_books", documents=documents, metadata=metadata, ids=ids, parallel=2)

Query the Qdrant database. This method runs a search query about `drama about people and unhappy love` and displays results.

It prints each result separated by a line of dashes, in the following format :

- Title: Title of the book
- Author: Author of the book
- Description: As stored in your document's description metadata field
- Published: Book publication date
- Score: Qdrant's relevancy score

In [34]:
results = qdrant.query(
    collection_name="my_books",
    query_text="drama about people and unhappy love",
    limit=2,
)
for result in results:
    print("Title:", result.metadata["title"], "\nAuthor:", result.metadata["author"])
    print("Description:", result.metadata["document"], "Published:", result.metadata["publishDate"], "\nScore:", result.score)
    print("-----")